In [ ]:
import cv2
import matplotlib.pyplot as plt
import sys
import os
import numpy as np

# --- Configuration: Determine Project Root and Adjust Paths/IDs as needed ---

current_executed_dir = os.getcwd()
# print(f"Current executed_dir (os.getcwd()): {current_executed_dir}")

# Determine the project root (which is the 'nutri-snap' folder)
if os.path.basename(current_executed_dir) == 'notebooks' and \
   os.path.basename(os.path.abspath(os.path.join(current_executed_dir, '..'))) == 'nutri-snap':
    project_root = os.path.abspath(os.path.join(current_executed_dir, '..'))
elif os.path.basename(current_executed_dir) == 'nutri-snap':
    project_root = current_executed_dir
else:
    # Fallback or error if the directory structure is not recognized
    print(f"WARNING: Unexpected directory structure. Attempting to use '{current_executed_dir}' as the nutri-snap project root.")
    print("Please run this script from the 'nutri-snap' directory or the 'nutri-snap/notebooks' directory.")
    project_root = current_executed_dir # Default assumption, may require manual adjustment

# print(f"Project root (nutri-snap folder): {project_root}")

# Add project root to sys.path to allow importing from src
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added {project_root} to sys.path")

# Try to import config and utils
try:
    from src.data_processing.utils import config
    from src.data_processing.utils.common_utils import load_category_info
    print("Successfully imported config and common_utils.")
except ImportError as e:
    print(f"Error importing from src.data_processing.utils: {e}")
    print("Make sure the project_root is correctly set and nutri-snap/src/data_processing/utils exists.")
    config = None # Ensure config is defined for later checks
    raise

# --- User Parameters ---
# YOU MUST MODIFY THESE to point to your actual data
SPLIT = 'train' # Or 'test', 'val', etc.
DISH_ID = 'dish_1558640849' # Change to your desired dish ID

# --- Path Definitions ---
# Assumes project_root is the 'nutri-snap' directory
absolute_base_processed_dir = os.path.join(project_root, 'data', 'processed')
# print(f"Absolute base processed dir: {absolute_base_processed_dir}")

semseg_image_path = os.path.join(absolute_base_processed_dir, SPLIT, 'semseg', DISH_ID, f'{DISH_ID}.png')
# print(f"Attempting to load semseg image from: {semseg_image_path}")

# --- Load N5K Category Information ---
n5k_int_id_to_name = {}
if config: # Proceed only if config was imported
    try:
        # Load N5K string ID to name mapping (e.g., "ingr_0000000001" -> "apple")
        n5k_str_id_to_name, _ = load_category_info(config.N5K_CATEGORY_TXT)

        # Create integer ID to name mapping (e.g., 1 -> "apple")
        if n5k_str_id_to_name:
            for str_id, name in n5k_str_id_to_name.items():
                if isinstance(str_id, str) and str_id.startswith("ingr_"):
                    try:
                        int_id = int(str_id.split('_')[-1])
                        n5k_int_id_to_name[int_id] = name
                    except ValueError:
                        print(f"Warning: Could not parse integer ID from N5K string ID '{str_id}'")
                elif isinstance(str_id, int): # If load_category_info parsed some IDs as int
                     n5k_int_id_to_name[str_id] = name
                # Add other parsing logic if N5K IDs in the text file might have other string formats 
                # that directly represent integers (e.g. "23" -> "carrot")
                elif isinstance(str_id, str) and str_id.isdigit():
                    try:
                        int_id = int(str_id)
                        n5k_int_id_to_name[int_id] = name
                    except ValueError:
                         print(f"Warning: Could not parse integer ID from N5K string ID '{str_id}'")


        if not n5k_int_id_to_name:
            print("Warning: n5k_int_id_to_name map is empty. Category names will not be available.")
        else:
            print(f"Loaded {len(n5k_int_id_to_name)} N5K integer ID to name mappings.")

    except AttributeError:
        print("Error: config.N5K_CATEGORY_TXT not found. Make sure config is correctly loaded.")
    except Exception as e:
        print(f"Error loading N5K category info: {e}")
else:
    print("Error: Config module not loaded. Cannot load N5K category information.")


# --- Load Semantic Segmentation Image ---
semseg_image_gray = cv2.imread(semseg_image_path, cv2.IMREAD_UNCHANGED) # Use IMREAD_UNCHANGED to preserve dtype (e.g., uint16)

if semseg_image_gray is None:
    print(f"Error: Could not load semantic segmentation image from {semseg_image_path}")
    # Create a placeholder if image loading fails
    semseg_image_gray = np.zeros((224, 224), dtype=np.uint8) 
    found_categories_str = "Image not found."
else:
    print(f"Semantic segmentation image loaded. Shape: {semseg_image_gray.shape}, Data type: {semseg_image_gray.dtype}")

    # --- Extract Unique N5K IDs and Map to Names ---
    unique_ids = np.unique(semseg_image_gray)
    # print(f"Unique pixel values (N5K IDs) in image: {unique_ids}")

    found_categories = []
    for uid in unique_ids:
        if uid == 0:  # Assuming 0 is background and should be ignored
            continue
        category_name = n5k_int_id_to_name.get(uid, f"Unknown ID: {uid}")
        found_categories.append(f"ID {uid}: {category_name}")
    
    if not found_categories:
        found_categories_str = "No N5K categories found (excluding background ID 0)."
    else:
        found_categories_str = "\n".join(found_categories)

# --- Display Image and Found Categories ---
plt.figure(figsize=(12, 6)) # Adjusted for side-by-side or stacked info

plt.subplot(1, 2, 1) # If you want to display text next to image
plt.imshow(semseg_image_gray, cmap='nipy_spectral') # 'nipy_spectral' or 'tab20' can be good for segmentation
plt.title(f"SemSeg for {DISH_ID} (Split: {SPLIT})")
plt.axis('off')

# Display found categories as text
plt.subplot(1, 2, 2)
plt.text(0.05, 0.95, "Found N5K Categories:\n---------------------\n" + found_categories_str, 
         fontsize=10, va='top', wrap=True)
plt.axis('off')

plt.suptitle(f"N5K Semantic Segmentation Analysis for {DISH_ID}", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make space for suptitle
plt.show()

if semseg_image_gray is not None: # Only print if image was loaded
    print("\nFound N5K Categories in Image:")
    print("-----------------------------")
    print(found_categories_str)
